In [1]:
%matplotlib inline

import os
import random
import time
import datetime
import copy
from decimal import Decimal, ROUND_HALF_UP
    
import matplotlib.pyplot as plt
import numpy as np
from netpyne import specs, sim
from neuron import h
import pandas as pd

### SETTINGS ##################

read_folder = 'Ground truth SWCs/Small'
simplify = False
SEED = 0
cells = 9
sigma_gnabar = 0.15
mu_gnabar = 0.37
netParams = specs.NetParams()
netParams.sizeZ = 10


# Estimated performance for the Octoscope
# https://iopscience.iop.org/article/10.1088/2040-8986/ac5dd5
# 500 Hz over a 95 µm × 477 µm FOV ~~> 500 Hz over a 212 µm x 212 µm FOV
# This is not that amazing for networks. So lets assume that future setups can do better:
netParams.sizeX = 500 # µm  
netParams.sizeY = 500 # µm        
microscope_recording_speed = 1000 # Hz
microscope_recording_duration = 0.150*1e3  #ms

impulse_delay = 10 #ms


################################

random.seed(SEED)
export_folder = f'dataset_s{SEED}_c{cells}'

if not os.path.exists(export_folder):
    os.makedirs(export_folder)

### Data storage arrays ##########
swc_filenames = []
for file in os.listdir(read_folder):
    if os.path.splitext(file)[-1] == '.swc':
        swc_filenames.append(file)

swcs = []
for ii in range(cells):
    swc_fname = random.choice(swc_filenames)
    swcs.append(swc_fname)

X = np.zeros(cells)
Y = np.zeros(cells)
Z = np.zeros(cells)

size = np.ceil(np.sqrt(cells)).astype(int)
for ii in range(cells):
    rel_coords = np.array([np.floor(ii / size), ii % size])
    rel_coords = rel_coords + 1
    rel_coords = rel_coords / (size + 2)

    X[ii] = netParams.sizeX * rel_coords[0]
    Y[ii] = netParams.sizeY * rel_coords[1]
    Z[ii] = netParams.sizeZ * 0.5
    
gnabars = np.random.normal(mu_gnabar, sigma_gnabar, cells)
###########################
        
cell_labels = []
for ii in range(cells):
    swc_fname = swcs[ii]
    
    label = f'trace_{ii:04d}'
    
    cellRule = netParams.importCellParams(
        label=label, 
        conds={'cellType': label},
        fileName=os.path.join(read_folder, swc_fname), 
        cellName=label,
        )
    
    cell_labels.append(label)
    
    # For convenience, we'll rename the first soma section in the morphology from `soma_0` to `soma`.
    netParams.renameCellParamsSec(label, 'soma_0', 'soma')
    
    
    for secName in cellRule['secs']:
        cellRule['secs'][secName]['geom']['cm'] = 1
        cellRule['secs'][secName]['mechs']['hh'] = {
            'gnabar': gnabars[ii], 
            'gkbar': 0.036, 
            'gl': 0.003, 
            'el': -70,
            }
        
    cellRule_dict = cellRule.todict()
    for jj, secName in enumerate(cellRule['secs']):
          
        parentX = cellRule_dict['secs'][secName]['topol'].get('parentX')
        childX = cellRule_dict['secs'][secName]['topol'].get('childX')
        
        if (parentX == 1.0 or parentX == 0.0) and (childX == 0.0):
            pass
        elif parentX == {} and childX == {}:
            pass #soma
        
        elif parentX is None and childX is None:
            pass 
        elif type(parentX) is float and type(childX) is float:
            # clip to 0-1, round up for 0.5
            cellRule['secs'][secName]['topol']['parentX'] = max(
                min(float(Decimal(parentX).to_integral_value(rounding=ROUND_HALF_UP)), 1.0), 
                0.0)
            
            # clip to 0-1
            cellRule['secs'][secName]['topol']['childX'] = max(
                min(float(round(childX)), 1.0), 
                0.0)

#             print(f"{parentX} --> {cellRule_dict['secs'][secName]['topol']['parentX']}\t" 
#                   f"{childX} --> {cellRule_dict['secs'][secName]['topol']['childX']}")
            
        else:
            raise ValueError('Expected parentX and childX relative positions to be floats between 0-1.'
                             f'\n Instead, got parentX: {parentX} and childX: {childX}')
    
    if simplify:
        # Postprocessing to allow for output
        for mm, secName in enumerate(cellRule['secs']):

            section_pts = cellRule['secs'][secName]['geom']['pt3d']

            if len(section_pts) > 1:
                old_pts = section_pts
                cellRule['secs'][secName]['geom']['pt3d'] = [old_pts[0], old_pts[1]] # Linear approx
    

    netParams.popParams[label] = {'cellType'  : label,
                                  'cellModel' : 'HH3D', 
                                  'numCells'  : 1,
                                  'xRange'    : [X[ii]-1, X[ii]+1], 
                                  'yRange'    : [Y[ii]-1, Y[ii]+1], 
                                  'zRange'    : [Z[ii]-1, Z[ii]+1], }
print(f'Added {ii+1} cells\n' 
    f'Used labels: {cell_labels}\n'
      f'Used {len(list(set(swcs)))} unique SWCs')


          
time.sleep(1)

Added 9 cells
Used labels: ['trace_0000', 'trace_0001', 'trace_0002', 'trace_0003', 'trace_0004', 'trace_0005', 'trace_0006', 'trace_0007', 'trace_0008']
Used 5 unique SWCs


In [2]:

## Synaptic mechanism parameters
netParams.synMechParams['exc'] = {'mod': 'Exp2Syn', 'tau1': 0.8, 'tau2': 5.3, 'e': 0}  # NMDA synaptic mechanism
netParams.synMechParams['inh'] = {'mod': 'Exp2Syn', 'tau1': 0.6, 'tau2': 8.5, 'e': -75}  # GABA synaptic mechanism

netParams.stimSourceParams[f'Probe'] = {'type': 'IClamp', 'del': impulse_delay, 'dur': 300, 'amp': 5}
for label in cell_labels:
    netParams.stimTargetParams[f'Probe->{label}'] = {'source': 'Probe', 
                                                 'sec': 'soma',
                                                 'loc': 0.5,
                                                 'conds': {'cellType': label}}

# stim_ii = random.randint(0, cells-1)
# print(f'Cell # {stim_ii} is stimulated')
# netParams.stimSourceParams['Probe'] = {'type': 'NetStim', 'rate': 30}
# netParams.stimTargetParams['Probe->Cell'] = {'source': 'Probe', 'conds': {'cellList': [stim_ii]}, 'weight': 0.5, 'delay': '10', 'synMech': 'exc'}

In [3]:
# Simulation options
simConfig = specs.SimConfig()        # object of class SimConfig to store simulation configuration

simConfig.duration = 0.150*1e3           # Duration of the simulation, in ms
simConfig.dt = 0.005                 # Internal integration timestep to use
simConfig.verbose = False            # Show detailed messages
simConfig.recordTraces = {'V_soma':{'sec':'soma','loc':0.5,'var':'v'}}  # Dict with traces to record
simConfig.recordCells = range(cells)
simConfig.recordStep = 0.25             # Step size in ms to save data (e.g. V traces, LFP, etc)

simConfig.analysis['plotTraces'] = {'oneFigPer' : 'trace' ,'saveFig': os.path.join(export_folder, 'traces.png')}  # Plot recorded traces for this list of cells
simConfig.analysis['plot2Dnet'] = {'saveFig': os.path.join(export_folder, 'network.png')}                                                # plot 2D cell positions and connections                                         # plot connectivity matrix
simConfig.analysis['plotShape'] = {
                                    'cvar'        : 'voltage', 
                                    'clim'        : [-70, -20], 
                                    'showFig'     : True,
                                    'elev'        : 45, 
                                    'azim'        : -45,
                                    'saveFig': os.path.join(export_folder, 'shape.png')
                                }
simConfig.analysis['plotSpikeStats'] = {'stats' : ['rate'],
                                        'saveFig': os.path.join(export_folder, 'spikes.png'),
                                       'saveData' : os.path.join(export_folder, 'spikes.json')}

# Create network and run simulation
sim.createSimulateAnalyze(netParams = netParams, simConfig = simConfig)

time.sleep(1)

#### Extra plot
data = []
for key, value in sim.simData['V_soma'].items():
    if key[0] is not '_':
        data.append(list(value))
        
data = np.array(data)
fig = plt.figure()
plt.imshow(data, aspect='auto', interpolation=None)
plt.xlabel('Time [ms]')
plt.ylabel('Neuron #')
plt.show()
plt.savefig(os.path.join(export_folder, 'traceimg.png'))

time.sleep(1)


Start time:  2022-06-21 11:56:45.792938

Creating network of 9 cell populations on 1 hosts...
  Number of cells on node 0: 9 
  Done; cell creation time = 0.02 s.
Making connections...
  Number of connections on node 0: 0 
  Done; cell connection time = 0.00 s.
Adding stims...
  Number of stims on node 0: 9 
  Done; cell stims creation time = 0.00 s.
Recording 9 traces of 1 types on node 0

Running simulation using NEURON for 150.0 ms...
  Done; run time = 1.53 s; real-time ratio: 0.10.

Gathering data...
  Done; gather time = 0.02 s.

Analyzing...
  Cells: 9
  Connections: 0 (0.00 per cell)
  Spikes: 96 (71.11 Hz)
  Simulated time: 0.1 s; 1 workers
  Run time: 1.53 s
Plotting recorded cell traces ... trace
Plotting 2D representation of network cell locations and connections...
Plotting 3D cell shape ...
Plotting spike stats...
Saving figure data as dataset_s0_c9\spikes.json ... 
  Done; plotting time = 1.20 s

Total time = 2.77 s


In [4]:


# time.sleep(1)


In [5]:
spike_times = np.array(sim.simData['spkt'])
spike_ids = np.array(sim.simData['spkid']).astype(int)

spike_freq = np.zeros(cells)

for tt, sid in zip(spike_times, spike_ids):
    spike_freq[sid] = spike_freq[sid] + 1

spike_freq =  spike_freq / (microscope_recording_duration) * 1000
    
print(len(np.unique(spike_freq)))

cells_data = {'swc':  swcs,
        'gnabar [S/cm^2]':  gnabars,
        'X [um]': X,
        'Y [um]': Y,
        'Z [um]': Z,
        'f [Hz]' : spike_freq,
        }

cells_df = pd.DataFrame(cells_data)
cells_df.to_csv(os.path.join(export_folder, 'cells.csv'))
          
settings = {'seed':  [SEED],
        'speed {Hz}' : microscope_recording_speed,
        'cells':  [cells],
        'FOV X size [um]': [netParams.sizeX],
        'FOV Y size [um]': [netParams.sizeY],
        }

settings_df = pd.DataFrame(settings)
settings_df.to_csv(os.path.join(export_folder, 'settings.csv'))

time.sleep(1)

6


In [6]:
if simplify:
    ref = f'network_{export_folder}'
    sim.createExportNeuroML2(netParams=netParams, 
                           simConfig=simConfig,
                           reference=ref)  # create and export network to NeuroML 2

In [ ]:
print('~~~SETTINGS FOR BlenderNEURON~~~\n')
print('Recording Settings:\n' 
      f'\tRecord Activity  : True\n'
      f'\tRecord from each : Section\n'
      f'\tStart Recording  : 0.00 \n'
      f'\tStop Recording   : {simConfig.duration}\n'
      f'\tRecord           : v \n'
      f'\tSampling Period  : {simConfig.recordStep}\n\n'
      f'\tFrames per\n' 
      f'\t     Millisecond : {microscope_recording_speed/1E3}\n\n'
      f'\tSimplification\n'
      f'\t     Tolerance   : 0.10\n\n'
      f'\tAnimate\n'
      f'\t     Brightness  : True\n\n'
      f'\tAnimate Color    : True\n'
      f'\tColor 0          : Hex 000000\n'
      f'\tColor 1          : Hex FFFFFF\n\n'
      f'\tVariable Low     : {np.floor(data.min())}\n'
      f'\tVariable High    : {np.ceil(data.max())}\n'
     )

print('NEURON:\n' 
      f'\tStop Time (ms)   : {simConfig.duration}\n'
      f'\tTemperature (°C) : 20 \n'
     )

print('~~~CHECK SCALE~~~\n')
middle_cell_label = cell_labels[cells//2]
middle_cell_soma_size = netParams.cellParams[middle_cell_label]['secs']['soma']['geom']['diam']
print(f'Soma diameter of middle cell : {middle_cell_soma_size:.2f}\n')

from blenderneuron import neuronstart

~~~SETTINGS FOR BlenderNEURON~~~

Recording Settings:
	Record Activity  : True
	Record from each : Section
	Start Recording  : 0.00 
	Stop Recording   : 150.0
	Record           : v 
	Sampling Period  : 0.25

	Frames per
	     Millisecond : 1.0

	Simplification
	     Tolerance   : 0.10

	Animate
	     Brightness  : True

	Animate Color    : True
	Color 0          : Hex 000000
	Color 1          : Hex FFFFFF

	Variable Low     : -71.0
	Variable High    : 50.0

NEURON:
	Stop Time (ms)   : 150.0
	Temperature (°C) : 20 

~~~CHECK SCALE~~~

Soma diameter of middle cell : 13.00

BlenderNEURON running in NEURON and accessible by Blender with BlenderNEURON addon at: http://127.0.0.1:56570


29.59891889917603
